<h1><center> PPOL 5203 Final Project <br><br> 
<font color='grey'>Python code Wendy's version<br><br>
Wendy Shi</center></center> <h1> 

# Acquiring NYT news info using API


In [1]:
import pandas as pd
import requests 
import dotenv
import os
from dotenv import load_dotenv
import time

import pandas as pd

API = pd.read_csv(r"/Users/wendyshi2001/Documents/APIs/NYT_api.csv")
my_API = API.iloc[0,1] #here's how to select and print specific value in pandas dataframe

In [2]:
all_state = ["Alabama","Alaska","Arizona","Arkansas","California",
             "Colorado","Connecticut","Delaware","Florida","Georgia",
             "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
             "Kansas", "Kentucky","Louisiana","Maine", "Maryland",
             "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri",
             "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
             "New Mexico","New York", "North Carolina", "North Dakota", "Ohio",
             "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
             "South Dakota", "Tennessee", "Texas", "Utah", "Vermont",
             "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]

In [3]:
for i in all_state:
    base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
    params = {
    'api-key': my_API,
    'q': 'pollution',
    'begin_date': '20010101',
    'end_date': '20231201',
    'fq': f'glocations:({i})', 
    #'news_desk': 'Environmnet',
    'page': 1
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        first_article = data['response']['docs']
        if len(first_article) == 10:
            #if there is 10 article in the list
            print(i,":", "data avaiable")
        else:
            print(i,":","no data available") 

    else:
        print("no connection")
    time.sleep(12)

Alabama : data avaiable
Alaska : data avaiable
Arizona : data avaiable
Arkansas : no data available
California : data avaiable
Colorado : data avaiable
Connecticut : data avaiable
Delaware : no data available
Florida : data avaiable
Georgia : data avaiable
Hawaii : data avaiable
Idaho : no data available
Illinois : no data available
Indiana : no data available
Iowa : data avaiable
Kansas : no data available
Kentucky : data avaiable
Louisiana : data avaiable
Maine : data avaiable
Maryland : data avaiable
Massachusetts : data avaiable
Michigan : data avaiable
Minnesota : no data available
Mississippi : no data available
Missouri : no data available
Montana : data avaiable
Nebraska : no data available
Nevada : no data available
New Hampshire : no data available
New Jersey : data avaiable
New Mexico : no data available
New York : no data available
North Carolina : data avaiable
North Dakota : data avaiable
Ohio : data avaiable
Oklahoma : data avaiable
Oregon : data avaiable
Pennsylvania : 

# Buildig a Scrapper

### Run this code: This function gives you the jason file of specific pages

In [4]:
# first define a function that changes page number
def fetch_nyt_data(state, page):
    base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'
    params = {
        'api-key': my_API,
        'q': 'pollution',
        'begin_date': '20010101',
        'end_date': '20231201',
        'fq': f'glocations:({state})', 
        #'news_desk': 'Environmnet',
        'page': page
        }
    response = requests.get(base_url, params=params)
    if response.status_code  == 200:
        response_json = response.json()
        
    return response_json


### Failed Code: Do not run!!!

def nytimes_state_data(state):
    
    jason_file = []
    
    for i in range(5) :
        data = fetch_nyt_data(state,i) #jason file is in the form of dictionary
        jason_file = jason_file + [data]
        time.sleep(12)
        
        #jason_file = f"{state}.data"
        #return f"{state}.data"
        return jason_file

# Colelcting data for each state

### This is the guideline that we follow collecting data for each state
-By changing range and wait time we are able to collect data for all states.

-For example, for CA that have 599 articles, we need to set wait time = 60 to pull 600 articles.

-This document serve as a example of how we collect data! We need to manually make adjustment collecting data for each state

### we manually collect data by states with 31 cva file

### Next, we manually convert them into one single cvs file

In [5]:
California = []
    
for i in range(10) :
    data = fetch_nyt_data("California",i) #jason file is in the form of dictionary
    California = California + [data]
    time.sleep(12)

In [6]:
len(California)

10

### 1: Collecting title

In [7]:
CA_tit = []
for i in range(len(California)):
    articles = California[i]['response']['docs']
    for article in articles:
        CA_tit = CA_tit + [article['headline']['main']]
print(len(CA_tit))
#CT_tit
#AL_tit

100


### 2: Collecting url

In [9]:
CA_url = []

for i in range(len(California)):
    articles = California[i]['response']['docs']
    for article in articles:
        CA_url = CA_url + [article['web_url']]

print(len(CA_url))

100


### 3: Collecting publish date

In [10]:
CA_time = []
for i in range(len(California)):
    articles = California[i]['response']['docs']
    for article in articles:
        CA_time = CA_time + [article["pub_date"]]
print(len(CA_time))


100


### 4: Collecting abstract

In [12]:
#The we collect all url
CA_ab = []

for i in range(len(California)):
    articles = California[i]['response']['docs']
    for article in articles:
        CA_ab = CA_ab + [article["abstract"]]
print(len(CA_ab))
#MD_ab


100


### 5: Sentiment by state

In [13]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
CA_senti = []

for i in CA_ab:
    if sentiment_pipeline(i)[0]["label"] == "POSITIVE":
        CA_senti = CA_senti + [1*sentiment_pipeline(i)[0]["score"]]
    else:
        CA_senti = CA_senti + [-1*sentiment_pipeline(i)[0]["score"]]

#MD_senti
CA_mean = sum(CA_senti)/len(CA_senti)
CA_mean

-0.4412294924259186

In [16]:
CA_column = {"Title": CA_tit,
             "URL" : CA_url,
              "Time": CA_time,
              "Abstrace": CA_ab}

CA_pd = pd.DataFrame(CA_column)

CA_pd["State"] = "Florida"
CA_pd.tail()

,Title,URL,Time,Abstrace,State
95,California Car Deal Criticized as Hurting Gree...,https://www.nytimes.com/2012/02/05/us/californ...,2012-02-04T22:44:02+0000,Critics say a provision of the Advanced Clean ...,Florida
96,California Leads on Warming,https://www.nytimes.com/2006/08/05/opinion/cal...,2006-08-05T05:00:00+0000,Editorial praises British Prime Min Tony Blair...,Florida
97,California Judge Calls Time Out for Climate Ch...,https://green.blogs.nytimes.com/2011/03/22/cal...,2011-03-22T22:13:58+0000,Conservatives spent millions trying to derail ...,Florida
98,L.A.’s Quest to Cut Fossil Fuels,https://www.nytimes.com/2016/10/12/opinion/las...,2016-10-12T07:21:21+0000,Los Angeles has the potential to serve as an e...,Florida
99,Last-Minute Damage to the Environment,https://www.nytimes.com/2003/12/12/opinion/las...,2003-12-12T05:00:00+0000,Editorial calls on Senate to review two damagi...,Florida


## Saving state data to a csv file

In [86]:
import pandas as pd

folder_path = "/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data" 
file_path = f'{folder_path}/FL_data.csv'

FL_pd.to_csv(file_path, index=False)

## Reload the state_file for concatenation

In [87]:
import pandas as pd

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/AK_data.csv'
AK_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/AL_data.csv'
AL_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/AZ_data.csv'
AZ_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/CA_data.csv'
CA_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/CO_data.csv'
CO_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/CT_data.csv'
CT_data = pd.read_csv(csv_file_path)

csv_file_path = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/FL_data.csv'
FL_data = pd.read_csv(csv_file_path)

In [165]:
print(AL_data.dtypes)

Title       object
URL         object
Time        object
Abstrace    object
State       object
dtype: object


In [90]:
Wendy_data = pd.concat([AK_data,AL_data,AZ_data,
                        CA_data,CO_data,CT_data,FL_data], axis=0)

In [91]:
folder_path = "/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data" 
file_path = f'{folder_path}/Wendy_data.csv'

Wendy_data.to_csv(file_path, index=False)

In [92]:
path1 = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/cyx_concat.csv'
CYX_data = pd.read_csv(path1)

path2 = '/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data/combined_states.csv'
TT_data = pd.read_csv(path2)

In [93]:
Full_data = pd.concat([Wendy_data,CYX_data,TT_data], axis=0)

In [94]:
folder_path = "/Users/wendyshi2001/Desktop/DataScience23/Final Project/CSV_data" 
file_path = f'{folder_path}/Full_data.csv'

Full_data.to_csv(file_path, index=False)